In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
from utilities import LogManager

### Using config.yaml

In [6]:
config_file = "./logger_config.yaml"
lm = LogManager(config_file)

Copy enabled (default behavior)
Signal handlers registered


Get Loggers

In [7]:
logger_A = lm.get_logger("logger_a")
logger_B = lm.get_logger("logger_b")

In [8]:
# Example use
logger_A.info("should only appear in console")
logger_A.critical("should appear in both console and file")
logger_B.debug("this should not appear since level for handler_console config is set to INFO")
logger_B.critical("should also appear in console, but for another task")

 2025-09-03 21:34:09 | INFO     | logger_a | 3623765976.py    | <module> : 2 - should only appear in console
 2025-09-03 21:34:09 | CRITICAL | logger_a | 3623765976.py    | <module> : 3 - should appear in both console and file
 2025-09-03 21:34:09 | CRITICAL | logger_b | 3623765976.py    | <module> : 5 - should also appear in console, but for another task


Assertion error is raised if an undefined logger is called.

In [ ]:
logger_c = lm.get_logger("logger_c")  # this should raise an assertion error, logger does not exist

Add new handler and logger

The following format (simple) is accepted as long as it's defined in config file under 'formats'

In [10]:
lm.add_handler(
    "handler_console_simple",
    {
        "sink": "sys.stdout",
        "format": "simple",
        "level": "info"
    }
)

Note that error will not be raised if format is not found in the config; only warning will be shown.

In [11]:
lm.add_handler(
    "handler_xxx",
    {
        "sink": "sys.stdout",
        "format": "simpleeee",
        "level": "info"
    }
)

 ⚠️ The format referenced by handler 'handler_xxx' is not defined in the 'formats' section of the config file. Using the format as is: 
	 simpleeee 



Because it may be a valid custom format:

In [12]:
lm.add_handler(
    "handler_console_fire",
    {
        "sink": "sys.stdout",
        "format": "🔥{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}",
        "level": "info",
    }
)

 ⚠️ The format referenced by handler 'handler_console_fire' is not defined in the 'formats' section of the config file. Using the format as is: 
	 🔥{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message} 



In [13]:
lm.add_logger("logger_c", [{"handler": "handler_console_fire", "level": "DEBUG"}])

In [14]:
logger_C = lm.get_logger("logger_c")
# Note: the effective logging level is the higher of the handler's and logger's levels.
# Therefore, DEBUG messages will not be logged because the handler's level is set to INFO.

logger_C.debug("this should not print since handler_console_fire is set to INFO")  # Will not be logged
logger_C.info("this should print")   # Will be logged
logger_C.error("this should print too")

🔥2025-09-03 21:39:50 | INFO     | logger_c | 2748549182.py    | <module> : 6 - this should print
🔥2025-09-03 21:39:50 | ERROR    | logger_c | 2748549182.py    | <module> : 7 - this should print too


Update handler

In [15]:
lm.update_handler(
    "handler_console_fire",
    {
        "sink": "sys.stdout",
        "format": "🧯{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}",
        "level": "debug",
    }
)

 ⚠️ The format referenced by handler 'handler_console_fire' is not defined in the 'formats' section of the config file. Using the format as is: 
	 🧯{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message} 



In [16]:
logger_C.debug("this should print now since handler and logger config is set to DEBUG")  # Will be logged
logger_C.info("this should print")
logger_C.error("this should print too")

🧯2025-09-03 21:41:01 | DEBUG    | logger_c | 1071904596.py    | <module> : 1 - this should print now since handler and logger config is set to DEBUG
🧯2025-09-03 21:41:01 | INFO     | logger_c | 1071904596.py    | <module> : 2 - this should print
🧯2025-09-03 21:41:01 | ERROR    | logger_c | 1071904596.py    | <module> : 3 - this should print too


Update logger

In [18]:
lm.update_logger("logger_c", [{"handler": "handler_console_fire", "level": "ERROR"}, {"handler": "handler_console", "level": "ERROR"}])

In [19]:
logger_C.debug("this should print since both logger config is set to ERROR")
logger_C.error("this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console")

 2025-09-03 21:42:38 | ERROR    | logger_c | 3130521273.py    | <module> : 2 - this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console
🧯2025-09-03 21:42:38 | ERROR    | logger_c | 3130521273.py    | <module> : 2 - this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console


Remove logger

In [20]:
lm.remove_logger("logger_c")
logger_C.debug("this should NOT appear on the console")
logger_C.info("this should NOT appear on the console")
logger_C.error("this should NOT appear on the console")
logger_A.info("only this should appear on the console")

 2025-09-03 21:43:46 | INFO     | logger_a | 2861226126.py    | <module> : 5 - only this should appear on the console


Remove handler

In [21]:
lm.remove_handler("handler_console")
logger_A.debug("this should NOT appear on the FILE.")
logger_A.info("this should NOT appear on the FILE.")
logger_A.error("this should appear on the FILE.")

### Start copy operations from config file

In [22]:
config_file = "./logger_config.yaml"
lm = LogManager(config_file)

Copy enabled (default behavior)
Signal handlers registered


In [23]:
lm.add_handler(
    "handler_file_2",
    {
        "sink": "./logs/subfolder/test2.log",
        "format": "simple",
        "level": "debug"
    }
)

In [24]:
lm.add_logger("logger_c", [{"handler": "handler_file_2", "level": "INFO"}])

In [25]:
logger_A = lm.get_logger("logger_a")
logger_B = lm.get_logger("logger_b")
logger_C = lm.get_logger("logger_c")

In [ ]:
logger_A.critical("test logger A")
logger_B.critical("test logger B")
logger_C.critical("test logger C")